In [88]:
import pandas as pd
import numpy as np
import json
import re

#Plotting
import matplotlib.pyplot as plt
import FCPython 

#Statistical fitting of models
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression


import os
import glob

In [89]:
xG_ext = pd.read_csv("xG_ext.csv",delimiter = ";", index_col=0)
xG_ext.head(12)

,Wk,Day,Date,Time,Home,xGHome,Score,xGAway,Away,Attendance,Venue,Referee,Match Report,Notes
Round,,,,,,,,,,,,,,
Bundesliga,1,Fri,2017-08-18,20:30,Bayern Munich,2.6,3–1,1.6,Leverkusen,"75,000",Allianz Arena,Tobias Stieler,Match Report,NaN
Bundesliga,1,Sat,2017-08-19,15:30,Hertha BSC,0.5,2–0,1.0,Stuttgart,"44,751",Olympiastadion Berlin,Sascha Stegemann,Match Report,NaN
Bundesliga,1,Sat,2017-08-19,15:30,Mainz 05,0.7,0–1,0.7,Hannover 96,"28,279",Opel Arena,Felix Brych,Match Report,NaN
Bundesliga,1,Sat,2017-08-19,15:30,Wolfsburg,0.4,0–3,1.0,Dortmund,"30,000",Volkswagen Arena,Robert Hartmann,Match Report,NaN
Bundesliga,1,Sat,2017-08-19,15:30,Hoffenheim,1.0,1–0,0.9,Werder Bremen,"30,150",Dietmar-Hopp-Stadion,Guido Winkmann,Match Report,NaN
Bundesliga,1,Sat,2017-08-19,15:30,Hamburger SV,0.5,1–0,0.8,Augsburg,"49,449",Volksparkstadion,Daniel Siebert,Match Report,NaN
Bundesliga,1,Sat,2017-08-19,18:30,Schalke 04,1.8,2–0,1.0,RB Leipzig,"61,435",Veltins-Arena,Felix Zwayer,Match Report,NaN
Bundesliga,1,Sun,2017-08-20,15:30,Freiburg,0.4,0–0,1.3,Eint Frankfurt,"24,000",Schwarzwald-Stadion,Manuel Gräfe,Match Report,NaN
Bundesliga,1,Sun,2017-08-20,18:00,Gladbach,1.5,1–0,1.1,Köln,"54,018",Stadion im Borussia-Park,Deniz Aytekin,Match Report,NaN


In [90]:
xG_ext = xG_ext[["Home", "Away", "xGHome", "xGAway"]].reset_index(drop=True)
xG_ext = xG_ext.dropna()
xG_ext = xG_ext[xG_ext["Home"]!="Home"].reset_index(drop=True)
xG_ext

,Home,Away,xGHome,xGAway
0,Bayern Munich,Leverkusen,2.6,1.6
1,Hertha BSC,Stuttgart,0.5,1.0
2,Mainz 05,Hannover 96,0.7,0.7
3,Wolfsburg,Dortmund,0.4,1.0
4,Hoffenheim,Werder Bremen,1.0,0.9
...,...,...,...,...
301,Hertha BSC,RB Leipzig,1.7,1.1
302,Hoffenheim,Dortmund,1.6,1.6
303,Bayern Munich,Stuttgart,2.1,2.2
304,Wolfsburg,Köln,1.9,1.2


## die Reihenfolge muss noch angepasst werden 

In [91]:
#dictionary aus Score DataFrame

df_scores = pd.read_csv('df_scores.csv', index_col =0)

teams = df_scores.team_home.unique()
teams = sorted(teams)
teams_dict = {}

for index, element in enumerate(teams):
    teams_dict[element] = index +1
 
 
teams_dictinv = {v: k for k, v in teams_dict.items()}
    
teams_dict

{'Augsburg': 1,
 'Bayer Leverkusen': 2,
 'Bayern München': 3,
 'Borussia Dortmund': 4,
 "Borussia M'gladbach": 5,
 'Eintracht Frankfurt': 6,
 'Freiburg': 7,
 'Hamburger SV': 8,
 'Hannover 96': 9,
 'Hertha BSC': 10,
 'Hoffenheim': 11,
 'Köln': 12,
 'Mainz 05': 13,
 'RB Leipzig': 14,
 'Schalke 04': 15,
 'Stuttgart': 16,
 'Werder Bremen': 17,
 'Wolfsburg': 18}

In [92]:
teams = xG_ext["Home"].unique()
sorted(teams)

['Augsburg',
 'Bayern Munich',
 'Dortmund',
 'Eint Frankfurt',
 'Freiburg',
 'Gladbach',
 'Hamburger SV',
 'Hannover 96',
 'Hertha BSC',
 'Hoffenheim',
 'Köln',
 'Leverkusen',
 'Mainz 05',
 'RB Leipzig',
 'Schalke 04',
 'Stuttgart',
 'Werder Bremen',
 'Wolfsburg']

In [93]:
extteams_dict = {'Augsburg': 'Augsburg',
 'Bayer Leverkusen': 'Leverkusen',
 'Bayern München': 'Bayern Munich',
 'Borussia Dortmund': 'Dortmund',
 "Borussia M'gladbach": 'Gladbach',
 'Eintracht Frankfurt': 'Eint Frankfurt',
 'Freiburg': 'Freiburg',
 'Hamburger SV': 'Hamburger SV',
 'Hannover 96': 'Hannover 96',
 'Hertha BSC': 'Hertha BSC',
 'Hoffenheim': 'Hoffenheim',
 'Köln': 'Köln',
 'Mainz 05': 'Mainz 05',
 'RB Leipzig': 'RB Leipzig',
 'Schalke 04': 'Schalke 04',
 'Stuttgart': 'Stuttgart',
 'Werder Bremen': 'Werder Bremen',
 'Wolfsburg': 'Wolfsburg'}


extteams_dictinv = {v: k for k, v in extteams_dict.items()}
extteams_dictinv

{'Augsburg': 'Augsburg',
 'Leverkusen': 'Bayer Leverkusen',
 'Bayern Munich': 'Bayern München',
 'Dortmund': 'Borussia Dortmund',
 'Gladbach': "Borussia M'gladbach",
 'Eint Frankfurt': 'Eintracht Frankfurt',
 'Freiburg': 'Freiburg',
 'Hamburger SV': 'Hamburger SV',
 'Hannover 96': 'Hannover 96',
 'Hertha BSC': 'Hertha BSC',
 'Hoffenheim': 'Hoffenheim',
 'Köln': 'Köln',
 'Mainz 05': 'Mainz 05',
 'RB Leipzig': 'RB Leipzig',
 'Schalke 04': 'Schalke 04',
 'Stuttgart': 'Stuttgart',
 'Werder Bremen': 'Werder Bremen',
 'Wolfsburg': 'Wolfsburg'}

In [94]:
xG_ext = xG_ext.replace(extteams_dictinv)
xG_ext = xG_ext.replace(teams_dict)
xG_ext

,Home,Away,xGHome,xGAway
0,3,2,2.6,1.6
1,10,16,0.5,1.0
2,13,9,0.7,0.7
3,18,4,0.4,1.0
4,11,17,1.0,0.9
...,...,...,...,...
301,10,14,1.7,1.1
302,11,4,1.6,1.6
303,3,16,2.1,2.2
304,18,12,1.9,1.2


In [95]:
df_scores

,team_home,team_away,ordresp
0,Bayern München,Stuttgart,5.0
1,Hoffenheim,Borussia Dortmund,1.0
2,Hertha BSC,RB Leipzig,5.0
3,Freiburg,Augsburg,1.0
4,Schalke 04,Eintracht Frankfurt,2.0
...,...,...,...
301,Hertha BSC,Stuttgart,1.0
302,Hamburger SV,Augsburg,2.0
303,Mainz 05,Hannover 96,4.0
304,Wolfsburg,Borussia Dortmund,5.0


In [96]:
df_scores = df_scores.replace(teams_dict)
df_scores

,team_home,team_away,ordresp
0,3,16,5.0
1,11,4,1.0
2,10,14,5.0
3,7,1,1.0
4,15,6,2.0
...,...,...,...
301,10,16,1.0
302,8,1,2.0
303,13,9,4.0
304,18,4,5.0


In [97]:
for index, match in df_scores.iterrows():
    xGhome = xG_ext[(xG_ext["Home"] == match["team_home"]) & (xG_ext["Away"] == match["team_away"])]["xGHome"]
    xGaway = xG_ext[(xG_ext["Home"] == match["team_home"]) & (xG_ext["Away"] == match["team_away"])]["xGAway"]
    #print(xGaway)
    
    
    df_scores.at[index, "xGhome"] = pd.to_numeric(xGhome.item())
    df_scores.at[index, "xGaway"] = pd.to_numeric(xGaway.item())
    
df_scores 

,team_home,team_away,ordresp,xGhome,xGaway
0,3,16,5.0,2.1,2.2
1,11,4,1.0,1.6,1.6
2,10,14,5.0,1.7,1.1
3,7,1,1.0,1.3,0.2
4,15,6,2.0,1.3,0.5
...,...,...,...,...,...
301,10,16,1.0,0.5,1.0
302,8,1,2.0,0.5,0.8
303,13,9,4.0,0.7,0.7
304,18,4,5.0,0.4,1.0


In [98]:
df_scores["goalDiffxG"] = df_scores["xGhome"] - df_scores["xGaway"]
df_scores

,team_home,team_away,ordresp,xGhome,xGaway,goalDiffxG
0,3,16,5.0,2.1,2.2,-0.1
1,11,4,1.0,1.6,1.6,0.0
2,10,14,5.0,1.7,1.1,0.6
3,7,1,1.0,1.3,0.2,1.1
4,15,6,2.0,1.3,0.5,0.8
...,...,...,...,...,...,...
301,10,16,1.0,0.5,1.0,-0.5
302,8,1,2.0,0.5,0.8,-0.3
303,13,9,4.0,0.7,0.7,0.0
304,18,4,5.0,0.4,1.0,-0.6


In [100]:
for i, row in df_scores.iterrows():
    if row['goalDiffxG'] <= -1.5:
        df_scores.at[i,"ordextxG"] = 5
    elif row['goalDiffxG'] <=-0.5:
        df_scores.at[i, 'ordextxG'] = 4
    elif row['goalDiffxG'] <0.5:
        df_scores.at[i, 'ordextxG'] = 3
    elif row['goalDiffxG'] <1.5:
        df_scores.at[i, 'ordextxG'] = 2
    elif row['goalDiffxG'] >=1.5:
        df_scores.at[i, 'ordextxG'] = 1
        
df_scores

,team_home,team_away,ordresp,xGhome,xGaway,goalDiffxG,ordextxG
0,3,16,5.0,2.1,2.2,-0.1,3.0
1,11,4,1.0,1.6,1.6,0.0,3.0
2,10,14,5.0,1.7,1.1,0.6,2.0
3,7,1,1.0,1.3,0.2,1.1,2.0
4,15,6,2.0,1.3,0.5,0.8,2.0
...,...,...,...,...,...,...,...
301,10,16,1.0,0.5,1.0,-0.5,4.0
302,8,1,2.0,0.5,0.8,-0.3,3.0
303,13,9,4.0,0.7,0.7,0.0,3.0
304,18,4,5.0,0.4,1.0,-0.6,4.0


In [101]:
ordextxG3 = pd.Series()
for i,j in df_scores.iterrows():
    if j["ordextxG"] <3:
        temp=1
    elif j["ordextxG"] == 3:
        temp=2
    else: 
        temp=3
    
    temp = pd.Series(temp)
    ordextxG3 = pd.concat([ordextxG3,temp])
ordextxG3 = ordextxG3.reset_index(drop=True)
ordextxG3

0      2
1      2
2      1
3      1
4      1
      ..
301    3
302    2
303    2
304    3
305    1
Length: 306, dtype: int64

In [102]:
df_scores["ordextxG3"] = ordextxG3
df_scores

,team_home,team_away,ordresp,xGhome,xGaway,goalDiffxG,ordextxG,ordextxG3
0,3,16,5.0,2.1,2.2,-0.1,3.0,2
1,11,4,1.0,1.6,1.6,0.0,3.0,2
2,10,14,5.0,1.7,1.1,0.6,2.0,1
3,7,1,1.0,1.3,0.2,1.1,2.0,1
4,15,6,2.0,1.3,0.5,0.8,2.0,1
...,...,...,...,...,...,...,...,...
301,10,16,1.0,0.5,1.0,-0.5,4.0,3
302,8,1,2.0,0.5,0.8,-0.3,3.0,2
303,13,9,4.0,0.7,0.7,0.0,3.0,2
304,18,4,5.0,0.4,1.0,-0.6,4.0,3


In [103]:
df_scores.to_csv("df_xG_ext.csv", index=False)

# Erstelle die Designmatrix für die Fälle k=5 und k=3

In [108]:
## k=5 ohne teamspezifischer Heimspieleffekt 

design_ext_xG = pd.read_csv("design_xG2_home.csv")
design_ext_xG =design_ext_xG.drop(design_ext_xG.columns[17], axis=1).copy()
design_ext_xG["ordextxG"] = df_scores["ordextxG"]

## k=3 ohne teamspezifischen Heimspieleffekt 

design_ext_xG3 = pd.read_csv("design_xG2_home.csv")
design_ext_xG3 =design_ext_xG3.drop(design_ext_xG3.columns[17], axis=1).copy()
design_ext_xG3["ordextxG"] = df_scores["ordextxG3"]



In [112]:
design_ext_xG

,GAMMA Augsburg,GAMMA Bayer Leverkusen,GAMMA Bayern München,GAMMA Borussia Dortmund,GAMMA Borussia M'gladbach,GAMMA Eintracht Frankfurt,GAMMA Freiburg,GAMMA Hamburger SV,GAMMA Hannover 96,GAMMA Hertha BSC,GAMMA Hoffenheim,GAMMA Köln,GAMMA Mainz 05,GAMMA RB Leipzig,GAMMA Schalke 04,GAMMA Stuttgart,GAMMA Werder Bremen,ALPHA,ordextxG
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,3.0
1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,3.0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,2.0
3,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2.0
4,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,4.0
302,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,3.0
303,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,1,3.0
304,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4.0


In [114]:
## das gleiche für den teamspezifischen Heimspieleffekt 
# k=5  
design_ext_xGts = pd.read_csv("design_xG2_spec.csv")
design_ext_xGts =design_ext_xGts.drop(design_ext_xGts.columns[35], axis=1).copy()
design_ext_xGts["ordextxG"] = df_scores["ordextxG"]

## k=3 

design_ext_xGts3 = pd.read_csv("design_xG2_spec.csv")
design_ext_xGts3 =design_ext_xGts3.drop(design_ext_xGts3.columns[35], axis=1).copy()
design_ext_xGts3["ordextxG"] = df_scores["ordextxG3"]


In [115]:
design_ext_xGts

,GAMMA Augsburg,GAMMA Bayer Leverkusen,GAMMA Bayern München,GAMMA Borussia Dortmund,GAMMA Borussia M'gladbach,GAMMA Eintracht Frankfurt,GAMMA Freiburg,GAMMA Hamburger SV,GAMMA Hannover 96,GAMMA Hertha BSC,...,ALPHA Hertha BSC,ALPHA Hoffenheim,ALPHA Köln,ALPHA Mainz 05,ALPHA RB Leipzig,ALPHA Schalke 04,ALPHA Stuttgart,ALPHA Werder Bremen,ALPHA Wolfsburg,ordextxG
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
1,0,0,0,-1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,3.0
2,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,2.0
3,-1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0
4,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,4.0
302,-1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3.0
303,0,0,0,0,0,0,0,0,-1,0,...,0,0,0,1,0,0,0,0,0,3.0
304,0,0,0,-1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.0


In [116]:
design_ext_xGts3

,GAMMA Augsburg,GAMMA Bayer Leverkusen,GAMMA Bayern München,GAMMA Borussia Dortmund,GAMMA Borussia M'gladbach,GAMMA Eintracht Frankfurt,GAMMA Freiburg,GAMMA Hamburger SV,GAMMA Hannover 96,GAMMA Hertha BSC,...,ALPHA Hertha BSC,ALPHA Hoffenheim,ALPHA Köln,ALPHA Mainz 05,ALPHA RB Leipzig,ALPHA Schalke 04,ALPHA Stuttgart,ALPHA Werder Bremen,ALPHA Wolfsburg,ordextxG
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,0,-1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
3,-1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,3
302,-1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
303,0,0,0,0,0,0,0,0,-1,0,...,0,0,0,1,0,0,0,0,0,2
304,0,0,0,-1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3


In [117]:
design_ext_xG.to_csv("design_ext_xG.csv", index=False)
design_ext_xG3.to_csv("design_ext_xG3.csv", index=False)
design_ext_xGts.to_csv("design_ext_xGts.csv", index=False)
design_ext_xGts3.to_csv("design_ext_xGts3.csv", index=False)
